# Get Meteo data and load into df

## Query structure:

api.meteomatics.com/ 2019-09-28T00:00:00Z P10D: P1D /t_2m:C/45.842875,6.863229/csv

2019-09-28T00:00:00Z - start date
P10D - time interval (10 days)
P1D - time step (1 day)



24hr max temp and 24hr total precipitation (https://www.meteomatics.com/en/api/available-parameters/basic-weather-parameter/):
http://api.meteomatics.com/2019-04-01T07:00:00ZP180D:P1D/t_max_2m_24h:C,precip_24h:mm/47.5596,7.5886/csv?model=mix


24hr mean temp and 24hr total precipitation:
http://api.meteomatics.com/2019-04-01T07:00:00ZP180D:P1D/t_mean_2m_24h:C,precip_24h:mm/47.5596,7.5886/csv?model=mix

1hr interval temps and 1hr precipitation:
http://api.meteomatics.com/2019-04-01T07:00:00ZP180D:P1H/t_mean_2m_1h:C,precip_1h:mm/47.5596,7.5886/csv?model=mix



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [13]:
df = pd.read_csv("./data/2019-09-27-basel-measures-FEAT.csv") #, keep_default_na = False) 
mt = pd.read_csv("./data/180_days_meanT_24hrPrecip.csv", sep=";")

In [14]:
df.head()

,osm_id,cci_id,date,place_name,place_type,cci,cci_p,hour,weekday,month,month_german,weekday_german,daytime
0,1175332462,287,2019-04-01 8:40:49,Leimgrubenweg,bus_stop,3.20000,3.20000,8,0,4,April,Montag,morning
1,25149740,86_9668,2019-04-01 8:40:53,Leimgrubenweg,secondary,3.35764,3.35764,8,0,4,April,Montag,morning
2,117485263,86_11641,2019-04-01 8:40:53,Reinacherstrasse,primary,3.35764,3.35764,8,0,4,April,Montag,morning
3,2621172927,287,2019-04-01 8:40:57,Leimgrubenweg,bus_stop,3.19757,3.19757,8,0,4,April,Montag,morning
4,148833576,86_12436,2019-04-01 8:42:11,Dornacherstrasse,secondary,5.00000,5.00000,8,0,4,April,Montag,morning


In [15]:
mt.head()

,validdate,t_mean_2m_24h:C,precip_24h:mm
0,2019-04-01T07:00:00Z,11.4,0.00
1,2019-04-02T07:00:00Z,11.2,0.00
2,2019-04-03T07:00:00Z,13.3,3.90
3,2019-04-04T07:00:00Z,5.7,6.90
4,2019-04-05T07:00:00Z,2.9,2.21


## Clean up the date format

In [22]:
mt["date_short"] = mt.validdate.apply(lambda x: x.split('T')[0])
mt.head()

,validdate,t_mean_2m_24h:C,precip_24h:mm,date_short
0,2019-04-01T07:00:00Z,11.4,0.00,2019-04-01
1,2019-04-02T07:00:00Z,11.2,0.00,2019-04-02
2,2019-04-03T07:00:00Z,13.3,3.90,2019-04-03
3,2019-04-04T07:00:00Z,5.7,6.90,2019-04-04
4,2019-04-05T07:00:00Z,2.9,2.21,2019-04-05


In [19]:
df["date_short"] = df.date.apply(lambda x: x.split(" ")[0])

## Join on `date_short`

In [24]:
df = pd.merge(df, mt, how="left", on="date_short")

In [25]:
df.head()

,osm_id,cci_id,date,place_name,place_type,cci,cci_p,hour,weekday,month,month_german,weekday_german,daytime,date_short,validdate,t_mean_2m_24h:C,precip_24h:mm
0,1175332462,287,2019-04-01 8:40:49,Leimgrubenweg,bus_stop,3.20000,3.20000,8,0,4,April,Montag,morning,2019-04-01,2019-04-01T07:00:00Z,11.4,0.0
1,25149740,86_9668,2019-04-01 8:40:53,Leimgrubenweg,secondary,3.35764,3.35764,8,0,4,April,Montag,morning,2019-04-01,2019-04-01T07:00:00Z,11.4,0.0
2,117485263,86_11641,2019-04-01 8:40:53,Reinacherstrasse,primary,3.35764,3.35764,8,0,4,April,Montag,morning,2019-04-01,2019-04-01T07:00:00Z,11.4,0.0
3,2621172927,287,2019-04-01 8:40:57,Leimgrubenweg,bus_stop,3.19757,3.19757,8,0,4,April,Montag,morning,2019-04-01,2019-04-01T07:00:00Z,11.4,0.0
4,148833576,86_12436,2019-04-01 8:42:11,Dornacherstrasse,secondary,5.00000,5.00000,8,0,4,April,Montag,morning,2019-04-01,2019-04-01T07:00:00Z,11.4,0.0


In [27]:
df=df.drop(df.columns[[13, 14]], axis=1)
df.head()

,osm_id,cci_id,date,place_name,place_type,cci,cci_p,hour,weekday,month,month_german,weekday_german,daytime,t_mean_2m_24h:C,precip_24h:mm
0,1175332462,287,2019-04-01 8:40:49,Leimgrubenweg,bus_stop,3.20000,3.20000,8,0,4,April,Montag,morning,11.4,0.0
1,25149740,86_9668,2019-04-01 8:40:53,Leimgrubenweg,secondary,3.35764,3.35764,8,0,4,April,Montag,morning,11.4,0.0
2,117485263,86_11641,2019-04-01 8:40:53,Reinacherstrasse,primary,3.35764,3.35764,8,0,4,April,Montag,morning,11.4,0.0
3,2621172927,287,2019-04-01 8:40:57,Leimgrubenweg,bus_stop,3.19757,3.19757,8,0,4,April,Montag,morning,11.4,0.0
4,148833576,86_12436,2019-04-01 8:42:11,Dornacherstrasse,secondary,5.00000,5.00000,8,0,4,April,Montag,morning,11.4,0.0


## Write out the df to .csv

In [30]:
df.to_csv("./data/2019-09-27-basel-measures-FEAT-TempPrecip.csv")

In [32]:
df = pd.read_csv("./data/2019-09-27-basel-measures-FEAT-TempPrecip.csv")

In [33]:
df.head()

,Unnamed: 0,osm_id,cci_id,date,place_name,place_type,cci,cci_p,hour,weekday,month,month_german,weekday_german,daytime,t_mean_2m_24h:C,precip_24h:mm
0,0,1175332462,287,2019-04-01 8:40:49,Leimgrubenweg,bus_stop,3.20000,3.20000,8,0,4,April,Montag,morning,11.4,0.0
1,1,25149740,86_9668,2019-04-01 8:40:53,Leimgrubenweg,secondary,3.35764,3.35764,8,0,4,April,Montag,morning,11.4,0.0
2,2,117485263,86_11641,2019-04-01 8:40:53,Reinacherstrasse,primary,3.35764,3.35764,8,0,4,April,Montag,morning,11.4,0.0
3,3,2621172927,287,2019-04-01 8:40:57,Leimgrubenweg,bus_stop,3.19757,3.19757,8,0,4,April,Montag,morning,11.4,0.0
4,4,148833576,86_12436,2019-04-01 8:42:11,Dornacherstrasse,secondary,5.00000,5.00000,8,0,4,April,Montag,morning,11.4,0.0
